## Importing libraries

In [1]:
#importing libraries needed for the model

import nltk
import nltk.data
from nltk.tokenize import TreebankWordTokenizer 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

#import qalsadi.analex as qa
#import pyarabic.araby as araby
#import naftawayh.wordtag
#from bs4 import BeautifulSoup

import re

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import os
#import json
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np

import re
from text_preprocessing import preprocess_text
from text_preprocessing import to_lower, lemmatize_word, expand_contraction
from text_preprocessing import remove_punctuation as punct


from collections import Counter
from nltk.corpus import stopwords
from scispacy.abbreviation import AbbreviationDetector

from collections import Counter



import gensim

import string

from gensim.models import Word2Vec

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cluster

from scipy.cluster import hierarchy

import spacy
nlp = spacy.load("en_core_web_sm")
from spacy.language import Language
abbreviation_pipe = AbbreviationDetector(nlp)
def create_lang_detector(nlp, name):
    return abbreviation_pipe
Language.factory("abbreviation_pipe", func=create_lang_detector)
nlp.add_pipe("abbreviation_pipe")


from abbreviations import schwartz_hearst
from nltk.corpus import wordnet
from textblob import TextBlob
import statistics as st


C:\Users\lenovo\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## reading in an input
## input is format of a string

In [2]:
#given a text extract key-value dictionary with sentenece as values and keys as their index in the list
# extract all words in the text into a list 
def read_input(text):
    
    lang_Dict = []
    for line in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(line):
            if word not in lang_Dict:
                lang_Dict.append(word)
    iter_dict = lang_Dict
    lang_Dict = []
    for i in iter_dict:
        if '\n' in i:
            lang_Dict = lang_Dict + i.split("\n")
        else:
            lang_Dict.append(i)
     
    remove = string.punctuation
    remove = remove.replace("-", "") # don't remove hyphens
    pattern = r"[{}]".format(remove)
    
    lang_dict = []
    for word in lang_Dict:
        if word.strip():
            word = word.lower()
            #word = expand_contraction(word)
            word = re.sub(pattern, '', word)
            lang_dict.append(word)
            
    lang_Dict = []
    for word in lang_dict:
        if " " in word:
            for w in word.split(" "):
                if w.strip():
                    lang_Dict.append(w)
        elif word and word.strip():
            lang_Dict.append(word)
    keyvalue = dict() 

    index= 0
    for line in nltk.sent_tokenize(text):
        if(line and (line.strip()) ):
            i = index
            keyvalue[i] = line.strip()
            index = index + 1
            
    key_value_list =[]
    for item in keyvalue.items():
        key_value_list.append(item)
    
    return keyvalue, key_value_list, lang_Dict

## necessary functions for preprocessing phase

In [3]:
#functions for the pre-processing phase

#this function rplaces the appreviation in the text 
#for example  Natural language processing --> NLP
def replace_acronymss(keyvalue):
        sent_dict = dict() 
        whole_text = ""
        for key, text in keyvalue.items():
            whole_text = whole_text + "#*# " + text
        whole_text = whole_text.lower()
        pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=whole_text, most_common_definition=True)
        for key,value in pairs.items():
            whole_text = whole_text.replace(value, key)
        output_text = whole_text.split("#*# ")
        i = 0;
        for l in output_text:
            if l != "":
                sentence = Brackets(l)
                sent_dict[i] = sentence
                i = i+1
        return sent_dict
    
#takes in words and expands contractions  
#for example expanding contractions won't --> will not 
#remove hyphens and tokenizing sentence
def remove_punctuation(words):
    new_words = []
    for word in words:
        word = expand_contraction(word)
        if word.lower() == "n't":
            word = "not"
        if word != '':
            word = word.lower()
            new_words.append(word)
    return new_words

#this function takes in a dictionary of values as sentence and the keys are their index in a sentnece 
#and splits the sentence into the list of words in it
#words that contain '\n-' in them are re-combined 
def tokenize_sent(sent_dict):
    keyvalueToken = dict() 
    TorF = True
    for key, value in sent_dict.items():
        
        words = nltk.word_tokenize(value)
        words = [re.sub(r'(\n-)','',word) for word in words]
        words = [re.sub(r'(\n)',' ',word) for word in words]
        new_words = []
        for word in words:
            if " " in word:
                 new_words = new_words + word.split(" ")
            else:
                new_words.append(word)
        
        keyvalueToken[key] = new_words
        
    return keyvalueToken

# this is a list of conjunctions to be removed from the tokenized sentence
# takes in a dictionary of values as tokenized sentences and keys as their index in the input document
# outputs a dictionary with same keys but values are tokenized sentences with conjunctions removed from them
def conjunctions(tokenize_sent_dict):
    new_sent_dict = dict()
    conjunctive_adverbs= ["accordingly","comparatively","contrarily", 
                      "also"," further", "nevertheless", "indeed"," otherwise",
                      "result","as","equally" ,"conversely" ,"besides",
                      "furthermore" ,"nonetheless" ,"fact" ,"consequently",
                     " likewise" ,"however" ,"addition" ",moreover" ,"surprisingly",
                     " hence" ,"similarly ","comparison" ,"still ","therefore ",
                      "contrast" ,"thus" ,"instead"  ,"rather"
                      , "other", "additionally", "addition",'although','already',"including",'according',
                    'totally','likely','unlikely','perhaps',"s",'naturally',
                      "finally", "based","on","that" , "besides","namely","anyway",
                     "then","next","thereafter","certainly","now","finally","meanwhile",
                     "subsequently","yet","elsewhere","thereafter","undoubtedly",
                     "incidentally","otherwise","regardless","begin","in"]
    
    for key, text in tokenize_sent_dict.items():
        new_sent = [word for word in text if word.lower() not in conjunctive_adverbs]
        new_sent_dict[key] = new_sent

    return new_sent_dict


# to remove brackets and everything in between
def Brackets(text):
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    return text

# given a sentence and a dictionary 
#check if the sentence is in this dictionary
#if it does return the key if not return "key doesn't exist"
def get_key(val,my_dict): 
    for key, values in my_dict.items(): 
        for value in values:
            if val == value: 
                return key 

    return "key doesn't exist"

# first iterate over sentences to get all synonym list in the document in a dictionary
# where the key is the first word in the text and the values are all it's synonyms that appeare in the text after it
def Synonym_replace(new_sent_dict):
    synonymList = []
    text = {}
    for key, sentence in new_sent_dict.items():
        for word in sentence:
            if word not in synonymList:
                wordNetSynset =  wn.synsets(word)
                values = []
                for synSet in wordNetSynset:
                    for synWords in synSet.lemmas():
                        if synWords.name() in synonymList:
                            continue
                        else:
                            synonymList.append(synWords.name())
                            values.append(synWords.name())
                text[word] = values
    return text
# then replace all synonms in the dictionary with their key  to unify all synonms
def replace_word_synonym(new_sent_dict):
    
    text = []
    sent_dict = dict()
    dict_synonym = Synonym_replace(new_sent_dict)
    for key, value in new_sent_dict.items():
        
        text = []
        for word in value:
            if get_key(word,dict_synonym) != "key doesn't exist":
                text.append(get_key(word,dict_synonym))
            else:
                text.append(word)
        sent_dict[key] = text
    
    return sent_dict

# to get the ngrames from a list of words
def ngramise(sequence):
    bigrams = []
    for bigram in nltk.ngrams(sequence, 2):
        bigrams.append(bigram)
    return bigrams

# removing the hyfen from word an checking if new words exist in the words of the document
#if they exist then the hyfen is removed if not then it is considered one word
def remove_hyfen(wordslist):
    words_new = []
    for word in wordslist:
        digit = False
        lang = False
        if "-" in word:
            words = word.split("-")
            for w in words:
                if w.isdigit():
                    digit = True
            if digit:
                newwords = [w for w in words]
                words_new = words_new + newwords
            else:
                for w in words:
                    if w in lang_Dict:
                        lang = True
                if lang:
                    newwords = [w for w in words]
                    words_new = words_new + newwords
                else:
                    newwords = [word]
                    words_new = words_new + newwords
        else:
            newwords = [word]
            words_new = words_new + newwords
    return words_new

# Lemmatize with POS Tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Word frequency computation
# remove stop words, punctuations and digits
#lemmatize text using wordnet lemmatizer with POS for better results
# output final list of sentences, list of words and their frequency, list of all words
def EF(sent_dict):
    final_sent = dict()
    all_words = []
    stopWords = set(stopwords.words('english'))
    #word_tokens = word_tokenize(text) 
    remove = string.punctuation
    remove = remove.replace("-", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    
    for key, value in sent_dict.items():
        value = remove_hyfen(value)
        value = remove_punctuation(value)
        filtered_sentence = [w for w in value if w not in stopWords] 
        #filtered_sentence = [re.sub(r'[" "]+', '', word) for word in filtered_sentence]
        #replace everything not in the list with space
        filtered_sentence = [re.sub("'", '', word.lower()) for word in filtered_sentence]
        filtered_sentence = [re.sub('"', '', word.lower()) for word in filtered_sentence]
        filtered_sentence = [re.sub(pattern, '', word) for word in filtered_sentence]
        filtered_sentence = [re.sub(r'[0-9]+', '', word) for word in filtered_sentence]
        filtered_sentence = [word for word in filtered_sentence if len(word.strip()) >1]
        filtered_sentence = [word for word in filtered_sentence if (re.sub(r"[^a-zA-Z-]+", '', word).strip())]
        #filtered_sentence = [re.sub(r"[^a-zA-Z-]+", '', word) for word in filtered_sentence]
        filtered_sentence = [word.strip() for word in filtered_sentence]

        wordnet_lemmatizer = WordNetLemmatizer() 
            
        filtered_sentence = [wordnet_lemmatizer.lemmatize(word,get_wordnet_pos(word))for word in filtered_sentence if  word != " " and word != '']
        
        final_sent[key] = filtered_sentence
        all_words = all_words + filtered_sentence
    word_frequency = list(Counter(all_words).items())
    return word_frequency, final_sent , all_words




    
    

## preprocessing function

In [4]:
#preprocess phase takes keyvalue dictionary after reading text an preprocess it
# word frequency in text, title, keywords
# outputs a list of sentences, and list of all words, dictionary containing a key-value list of sentences for
# the processing phase


def preprocess():
    
    #detection of appreviation
    sent_dict = replace_acronymss(keyvalue)

    tokenize_sent_dict = tokenize_sent(sent_dict)
    
    new_sent_dict = conjunctions(tokenize_sent_dict)
    
    #e Replacement of word synonyms
    sent_dict = replace_word_synonym(new_sent_dict)
    sent_list =[]
    for item in sent_dict.items():
        sent_list.append(item)

    all_bigrams = []
    for key, value in sent_dict.items():
        bigrams = ngramise(value)
        all_bigrams = all_bigrams+bigrams
    #only keep bi-grams that appear atleast twice
    bi_count = dict(Counter(all_bigrams))
    delete = [key for key,value in bi_count.items() if value < 2]
    # delete the key 
    for key in delete: del bi_count[key]
    bi_count_list =[]
    for key , value in bi_count.items():
        bi_count_list.append([(key),value])
        
    
    word_freq ,final_sent, all_words = EF(sent_dict)
    Sentences_list = []
    for item in final_sent.items():
        Sentences_list.append(item)     
    
    #word frequencies
    #freq of words in title
    title =[]
    title_freq = Counter(title)

    keywords =[]
    Keyword_freq = Counter(keywords)
    #Keyword_freq = keywords
    #print(Keyword_freq)
    w_freq = Counter(all_words)
    final_Word_freq = dict()
    for word in all_words:
        if word not in title_freq.keys():
            title_freq[word] = 0
        if word not in Keyword_freq.keys():
            Keyword_freq[word] = 0
        final_Word_freq[word] = title_freq[word] + Keyword_freq[word] + w_freq[word] 

    ##final list of sentences after preprocessing
    final_input = []
    for (index, sentence) in Sentences_list:
        final_input.append(" ".join(sentence))
        
    keyword_word_freq = []
    for word in all_words:
        keyword_word_freq.append([word,Keyword_freq[word]])  
    
    title_word_freq = []
    for word in all_words:
        title_word_freq.append([word, title_freq[word]])

    Sentences_list
    List_of_sent = dict()
    for (key, sent) in Sentences_list:
        List_of_sent[key] = " ".join(sent)

    final_Word_freq
    finalwordfreq = []
    for key , value in final_Word_freq.items():
        finalwordfreq.append([key,value])
    
    return (Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
            ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq)

## function to get the frequency of words in bigrams

In [5]:
# takes in list of bi-grams and outputs a list of (word, frequency) 
def bigrams_freq(bi_count_list):
    words_freq_bigram = dict()
    bi_count = []
    for list_bi in bi_count_list:
        (w1,w2), i = list_bi
        if w1 in words_freq_bigram.keys():
            words_freq_bigram[w1] = words_freq_bigram[w1] + i
        else:
            words_freq_bigram[w1] = i
        if w2 in words_freq_bigram.keys():
            words_freq_bigram[w2] = words_freq_bigram[w2] + i
        else:
            words_freq_bigram[w2] = i   
    for key,value in words_freq_bigram.items():
        bi_count.append([key,value])
    return bi_count

## Weight computation function

In [6]:
# takes in a list of nodes and outputs their weights using their frequencies
#weights
def get_weights(list_of_nodes):
    import pandas as ps
    import statistics as st
    node_weight_dict = dict()
    node_list= list_of_nodes
    FWL= finalwordfreq
    TWL= title_word_freq
    KWL= keyword_word_freq
    BWL= bi_count 
    PWL= Pnouns
    #BWL = []
    #PWL = []
    

    F=[]     #freq of words  which appeared in nodes WRT document
    T=[]     #freq of words  which appeared in nodes WRT title
    K=[]     #freq of words  which appeared in nodes WRT keyword list 
    B=[]     #freq of words which appeared in nodes WRT Bi_Gram list 
    P=[]     #list of detection the proper noune 
    FW=[]    
    Xt,Xk,Xbi,Xp,Xb=1,1,1,1,1
    weighted_node=[]
    
    
    for i in node_list:
        flag=0
    
        for j in FWL:
            if(i==j[0]):
                f=j[1]
                F.append(f)
                flag=1
                
        if(flag!=1): 
            F.append(0) 
        
 

    for i in node_list:
        flag=0
        for j in TWL:
            if(i==j[0]):
                t=j[1]
                T.append(t)
                flag=1
        if(flag!=1): 
             T.append(0)
                
        
    for i in node_list:
        flag=0
        for j in KWL:
            if(i==j[0]):
                k=j[1]
                K.append(k)
                flag=1
        if(flag!=1): 
            K.append(0)
            
            
            
    for i in node_list:
        flag=0
        for j in BWL:
            if(i==j[0]):
                b=j[1]
                B.append(b)
                flag=1
        if(flag!=1): 
            B.append(0)
            
            
    for i in node_list:
        flag=0
        for j in PWL:
            if(i==j):
                P.append(1)
                flag=1
        if(flag!=1): 
            P.append(0)             
    
    
    
    for i in FWL:
        fw=i[1]
        FW.append(fw)
    
    if FW == []:
        avrage_word_frequency= 0
        median_word_frequency= 0 
    elif len(FW) != 0:
        avrage_word_frequency=st.mean(FW)
        median_word_frequency=st.median(FW) 
    else:
        avrage_word_frequency= 0
        median_word_frequency= 0 
        
    AW=abs(avrage_word_frequency-median_word_frequency)
    
    for i in range(len(node_list)):
        WN=F[i]+Xt*AW*T[i]+Xk*AW*K[i]+Xb*AW*B[i]+Xp*AW*P[i]
        weighted_node.append(WN)
        node_weight_dict[node_list[i]] = WN
    
    return node_weight_dict,weighted_node
    

## functions for algorithm two Getting the candidate edge

In [7]:
#edge between source and distenation
def get_edge(source,distenation):
    return edgeget[source,distenation]

#distination node list
# gets the weights of the distination nodes given a source node , all node weights and source-destination dictionary 
def getweights_of_dist_nodes(source_destinations,sourcenode,All_nodes_ww):  
    x=source_destinations[sourcenode]
    distenation_node_list={}
    weights=[]
    nothing=0
    for i in range(len(x)):
        if ((x[i])==('E#')) or ( x[i] not in All_nodes_ww):
            #nothing is a flag for destination nodes
            #if one then there is none
            nothing=1
        else:
            distenation_node_list[x[i]]=All_nodes_ww[x[i]]
            weights.append((All_nodes_ww[x[i]]))
        if len(weights) != 0:
            nothing=0
    return distenation_node_list,weights,nothing



## algorithm two

In [8]:
#check for candidate edges between a source and a destination given a criteria 
def get_candidate_edges(source_destinations,node_listx,wxx,out_carteria,All_weight,input_nodes,All_SD):
    c=[]
    xxx = []
    no_list,totalweights=node_listx,All_weight

    source_node_weight=0
    distination_node_weight=0
    average_of_the_node_weight=np.average(totalweights)#(np.sum(node_list)/number_of_nodes)
    median_of_the_node_list_weight=np.median(totalweights)
  
    if average_of_the_node_weight>median_of_the_node_list_weight:#can be replaced with source=max(avg,median)
        source_node_weight=average_of_the_node_weight
    else:
        source_node_weight=median_of_the_node_list_weight

    for node,weight in no_list.items():
        
        if weight>=source_node_weight:#بيعرف اذا كانت بدايه الجمله ولا لا 
            outnodes1,outnodesweights,flagnothing=getweights_of_dist_nodes(source_destinations,node,no_list)#فنكشن جوا الجي بتجبلي كل الديستناشن بتاعت النودواختار بينهم هسقط مين
            outnodes2,nodes2weights,flag2nothing=getweights_of_dist_nodes(All_SD,node,input_nodes)
            outnodes1=[(i,j) for (i,j) in outnodes1.items() if j!=0 ] 
            outnodes2 =[(i,j) for (i,j) in outnodes2.items() if j!=0 ] 
        
            if flagnothing==0:
                #destination nodes exsist
                #get their average and max weights for avg and max criterias
                average_weight=np.average(nodes2weights)
                max_weight=max(nodes2weights)
            else:
                #no destination nodes 
                average_weight=0
                max_weight=0
            # get threshold of destination weights according to criteria 
            if (out_carteria =="avg"):
                distination_node_weight=average_weight
            elif (out_carteria =="node_avg"):
                distination_node_weight=source_node_weight
            elif (out_carteria=="max"):
                distination_node_weight=max_weight
            else:
                distination_node_weight=average_weight


            if (bool(outnodes1)==False):
                # no destination nodes 
                nothing=1  
            else: 
                # check for the candidate edges
                for out,weight_of_out_node in outnodes1:
                    if weight_of_out_node>=distination_node_weight and flagnothing==0:
                        c.append(get_edge(node,out))# append the edge
                        xxx.append([node,out])

    cx=[]
    for i in range(0,len(c)):
        cx.append(c[i][0])
    return cx
  

## functions for the post processing phase (algorithm 4)

In [9]:
# get the weight of the sentence order in the input document
def sentence_order(count_summ, index):
    return(count_summ - index)/count_summ

#given a list of candidate sentences rank them according to ranking criteria
def compute_rank(candidate_summary):
    ranks = {}
    candidate_summary = reorder(candidate_summary)
    
    for i, sent in enumerate(candidate_summary):
        rank = sentence_order(len(candidate_summary), i)
        ranks[sent] = rank
        
    sort_ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)    
    return ranks   

#vectorize sentences for clustering
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass
     
    return np.asarray(sent_vec) / numw

#numner of words in a sentence
def get_words_count(sent):
    len_sent = 0
    for i in sent.split(" "):
        len_sent = len_sent +1
    return len_sent

# reorder the summary based on the order from the input text
def reorder(summary):
    index_summ = []
    key_sent = list(keyvalue.keys())
    value_sent = list(keyvalue.values())
    order = dict()
    final_order = []
    for sent in summary:
        if sent in value_sent:
            index_sent = value_sent.index(sent)
            index_summ.append(index_sent)
            order[index_sent] = sent
            
    for i in sorted(order):
        final_order.append(order[i])

    return final_order

#concatenate the sentences in the candidate summary
def concat(final_order):
    return("\n".join(final_order))
 
# get the index of a sentence from the input text
def get_index(sent):
    key_sent = list(keyvalue.keys())
    value_sent = list(keyvalue.values())
    index_sent = value_sent.index(sent)
    return index_sent  


## Ranking the sentences and preparing for clusting

In [10]:
#rank sentences then cluster them in 5 clusters
def post_processing(list_of_candidates_summeries):
    global n_clusters
    global cluster_map
    candidate_summary = list_of_candidates_summeries
    sentences_ranks = compute_rank(candidate_summary)
   
    sentences_ordered =  [key for key,value  in sentences_ranks.items()]
    summary_length =0
    visited_sentences = []
    
    sentences = sentences_ordered

    tfidfvect = TfidfVectorizer(stop_words='english')
    X = tfidfvect.fit_transform(sentences)

    # number of clusters is one for no clusting
    if len(sentences) <1: # number of clusters can't be more than number of sentences
        n_clusters = len(sentences)
    else:
        n_clusters = 1
    clf = KMeans(n_clusters = n_clusters,init = 'k-means++', max_iter = 1000)
    labels = clf.fit_predict(X)
    clustered_sentences =[[] for i in range(5)]

    cluster_map = pd.DataFrame()
    cluster_map['data_index'] = sentences
    cluster_map['cluster'] = clf.labels_
    #print(cluster_map) to visualize clustered sentences
    return n_clusters,cluster_map

## Algorithm 4 (clustring) 

In [11]:
def algorithm_4(summary_length,threshold,max_words_limit,list_of_candidates_summeries,n_clusters,cluster_map):
    candidate_summary = list_of_candidates_summeries
    summary_length = 0
    while (len(visited_sentences) < len(candidate_summary)) and ((summary_length)< max_words_limit) :
        for i in range(n_clusters):
        
            clustered_sentences = cluster_map[cluster_map.cluster == i]['data_index'].tolist()
            
            if summary_length >= (max_words_limit+threshold):
                #if we reached the limit break
                break
            for sent in clustered_sentences:
                if sent not in visited_sentences :
                    #check sentences in the cluster that was not added to candidate sentences
                    temp_length = summary_length + get_words_count(sent)
                    if temp_length < max_words_limit :
                        summary_length = temp_length
                        final_summary.append(sent)
                        #if the length of summary doesn't exceed the maximum length add the sentence to the final summary
                        visited_sentences.append(sent)
                        break
                    elif(temp_length >= max_words_limit) and (temp_length <= (max_words_limit + threshold)):
                        #else check if it is less than the max length + a given threshold 
                        summary_length = temp_length
                        final_summary.append(sent)
                        visited_sentences.append(sent)
                        break
                    elif temp_length > (max_words_limit + threshold):
                        #else this sentence can't be added to the final summary as it exceeds  the length and the threshold
                        visited_sentences.append(sent)
        
    return final_summary


## functions for the graph algorithm (algorithm 1)

In [12]:
# creating the  graph

# Add a vertex to the dictionary
def add_vertex(v):
    global graph
    global vertices_no
    if v not in graph:
        vertices_no = vertices_no + 1
        graph[v] = []

# Add an edge between vertex v1 and v2 with edge weight e
#v1 is the source node and v2 is the distenation node
edgeget={}
def add_edge(v1, v2, e_order, e_value):
    global graph
    # Check if vertex v1 is a valid vertex
    if ((v1 in graph) and (v2 in graph)):
        temp = [v2, e_order, e_value]
        edgeget[v1,v2]=[e_value]
        graph[v1].append(temp)

# Print the graph
def print_graph():
    global graph
    for vertex in graph:
        for edges in graph[vertex]:
            print(vertex, " -> ", edges[0], "    edge order:", edges[1], "    edge value:", edges[2])

# get the destinations
def get_destinations(source):
    global graph
    distinations=[]
    if(source in graph):
        for elements in graph.get(source):
            if(elements[0] not in distinations):
                distinations.append(elements[0])
    return distinations

# get the edges
def get_edges(source,dist):
    global graph
    Edge=""
    if(source in graph and dist in graph):
        for elements in graph.get(source):
            if(elements[0]==dist):
                Edge= Edge + elements[2]
    return Edge


## The iterative part of the algorithm 
##  iterates over candidate summary until it stabilize 

In [16]:
def iter_graph(Cand_edge,can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria):
   
    All_SD = {}
    input_nodes = []
    sent_nodes = {}      
    nodes_list = []
    input_text = candidate_summ
    last_summ_len = candidate_summ_len
    Last_summ = candidate_summ
    sent_idx = can_edges
    can_edges=[]
    List_of_sent = list_pre_cansumm
    list_pre_cansumm = []
    list_of_candidates_summeries = []
    All_nodes=[]
    nodes_list = []
    # driver code
    graph = {} 
    # stores the number of vertices in the graph
    vertices_no = 0    
    S = List_of_sent
    source_destinations={}
    index = 0
    
    for sent_index, sentence in zip(sent_idx,List_of_sent):
        nodes_list = ["S#"]
        edges_list = [""]
        counter =0
        edge_value =""

        Words=tokenizer.tokenize(sentence)

        Types=pos_tag(Words)
       ## check for both nouns and verbs
        for Type in Types:
            if (Type[1]=='NN' or Type[1]=='NNP' or Type[1]=='NNS' or Type[1]=='NNPS' or Type[1]=='VB' or Type[1]=='VBG' or Type[1]=='VBD' or Type[1]=='VBZ'):
                nodes_list.append(Type[0])
                edges_list.append("")
            else:
                edges_list.append(Type[0])
                nodes_list.append("")



        list_of_nodes =[]
        LIST_OF_NODES=[]
        for i in nodes_list:
            if(i != ""):
                add_vertex(i)
                list_of_nodes.append(i)
                final_node =i
        list_of_nodes.remove("S#")    
        add_vertex("E#")
        nodes_list.append("E#")
        edges_list.append("")

        counter =0
        order =0
        
        for j in nodes_list :
            if(nodes_list[counter+1] !=""):
                add_edge(nodes_list[counter], nodes_list[counter+1], order, "")
                order =order +1
            else:
                x= counter
                y= edges_list[counter+1]
                while(nodes_list[counter+1] =="" and nodes_list[x] !=""):
                    counter =counter +1
                    if(counter ==len(nodes_list)-2):
                        y= y+" "+ edges_list[counter+1]
                add_edge(nodes_list[x], nodes_list[counter+1], order, y)
                order =order +1
            counter =counter +1
            if(counter == len(nodes_list)-1):
                break
        
        for k in list_of_nodes:
            if(k not in LIST_OF_NODES):
                LIST_OF_NODES.append(k)


        for s in LIST_OF_NODES:
            if(s not in source_destinations.keys()):
                source_destinations[s]=get_destinations(s)
            else:
                source_destinations[s]= source_destinations[s]+ get_destinations(s)
            
        for s in LIST_OF_NODES:
            if(s not in All_SD.keys()):
                All_SD[s] = get_destinations(s)
            else:
                All_SD[s] = All_SD[s] + get_destinations(s)
        
        #****************************************************************************
        #all nodes in the sentence
        input_nodes = input_nodes + list_of_nodes
        sent_nodes[index] = (source_destinations,list_of_nodes) #nodes in a sentences
    
        #****************************************************************************
        graph = {}
        source_destinations={}
        index = index + 1
    input_nodes = list(set(input_nodes))
    dic_of_nodes_and_weights,list_of_weights=get_weights(input_nodes) # getting the weights of all nodes in a sentence
    All_weight = list_of_weights
    
    index = 0
    for sent_index, sentence in enumerate(List_of_sent): 
        (source_destinations,list_of_nodes) = sent_nodes[index]
        nodes_weights = {}
        weights = []
        for node in list_of_nodes:
            nodes_weights[node] = dic_of_nodes_and_weights[node]
            weights.append(dic_of_nodes_and_weights[node])


        list_of_can_edges=get_candidate_edges(source_destinations,nodes_weights,weights,out_carteria,All_weight,dic_of_nodes_and_weights,All_SD)
        if len(list_of_can_edges)>=1:#we will make it a variable depends on the length of the sentence
            if(List_of_sent[index] and (List_of_sent[index].strip()) ):
                list_of_candidates_summeries.append(keyvalue[sent_index])
                list_pre_cansumm.append(List_of_sent[index])
                can_edges.append(sent_index)

        index = index + 1

    candidate_summ = reorder(list_of_candidates_summeries)
    candidate_summ_len = 0
    for index ,sent in zip(sent_idx,List_of_sent):
        candidate_summ_len = candidate_summ_len + get_words_count(sent)

    return list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges

## final summarization function
## main function takes in an input string and outputs a summary

In [14]:
#this wraps up all of the model 
#input a text
#output a summary
def summarization(Text_in):

    global vertices_no
    global keyvalue
    global key_value_list
    global lang_Dict
    global Sentences_list
    global final_input
    global all_bigrams
    global bi_count_list
    global word_freq
    global all_words
    global finalwordfreq
    global List_of_sent
    global title_word_freq
    global keyword_word_freq
    global All_nodes
    global list_of_candidates_summeries
    global list_pre_cansumm
    global Word_Freq
    global bi_count
    global Cand_edge
    out_carteria="avg"
    All_SD = {}
    input_nodes = []
    sent_nodes = {}      
    nodes_list = []
    vertices_no =0
    can_edges = []

    keyvalue, key_value_list,lang_Dict = read_input(Text_in) 
    
    (Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
     ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq) = preprocess() 

    Cand_edge =[]
    Word_Freq = {}
    for key, value in finalwordfreq:
        Word_Freq[key] = value
    
    bi_count = bigrams_freq(bi_count_list) 

    Last_summ = keyvalue.values()
    last_summ_len = 0
    for value in keyvalue.values():
        last_summ_len = last_summ_len + get_words_count(value)


    All_nodes=[]
    list_of_candidates_summeries=[]
    list_pre_cansumm = []

    tokenizer = TreebankWordTokenizer()

    ##lemmatization
    lemmatizer = WordNetLemmatizer() 

    # Add an edge between vertex v1 and v2 with edge weight e
    #v1 is the source node and v2 is the distenation node
    edgeget={}

    
    # driver code
    global graph 
    graph = {} 
    # stores the number of vertices in the graph
    vertices_no = 0    
    S = List_of_sent
    source_destinations={}
    input_nodes = []
    sent_nodes = {}
    global Pnouns
    Pnouns = []

    for sent_index, sentence in enumerate(List_of_sent.values()):
        nodes_list = ["S#"]
        edges_list = [""]
        counter =0
        edge_value =""
        
        Words=tokenizer.tokenize(sentence)
        
        Types=pos_tag(Words)
        for Type in Types:
            if (Type[1]=='NN' or Type[1]=='NNP' or Type[1]=='NNS' or Type[1]=='NNPS' or Type[1]=='VB' or Type[1]=='VBG' or Type[1]=='VBD' or Type[1]=='VBZ'):
                nodes_list.append(Type[0])
                edges_list.append("")
                if(Type[1] == 'NNP'):
                    Pnouns.append("")
            
            else:
                edges_list.append(Type[0])
                nodes_list.append("")
 

        list_of_nodes =[]
        for i in nodes_list:
            if(i != ""):
                add_vertex(i)
                list_of_nodes.append(i)
                final_node =i

        list_of_nodes.remove("S#")    
        add_vertex("E#")
        nodes_list.append("E#")
        edges_list.append("")

        counter =0
        order =0
        
        for j in nodes_list :
            if(nodes_list[counter+1] !=""):
                add_edge(nodes_list[counter], nodes_list[counter+1], order, "")
                order =order +1
            else:
                x= counter
                y= edges_list[counter+1]
                while(nodes_list[counter+1] =="" and nodes_list[x] !=""):
                    counter =counter +1
                    if(counter ==len(nodes_list)-2):
                        y= y+" "+ edges_list[counter+1]
                add_edge(nodes_list[x], nodes_list[counter+1], order, y)
                order =order +1
            counter =counter +1
            if(counter == len(nodes_list)-1):
                break

        LIST_OF_NODES=[]
        for k in list_of_nodes:
            if(k not in LIST_OF_NODES):
                LIST_OF_NODES.append(k)
        
        for s in LIST_OF_NODES:
            if(s not in source_destinations.keys()):
                source_destinations[s]=get_destinations(s)
            else:
                source_destinations[s]= source_destinations[s]+ get_destinations(s)

        for s in LIST_OF_NODES:
            if(s not in All_SD.keys()):
                All_SD[s] = get_destinations(s)
            else:
                All_SD[s] = All_SD[s] + get_destinations(s)

        #****************************************************************************
        input_nodes = input_nodes + list_of_nodes
        sent_nodes[sent_index] = (source_destinations,list_of_nodes)

        #****************************************************************************
        graph = {}
        source_destinations={}
        print_graph() 
        #****************************************************************************     
    input_nodes = list(set(input_nodes))
    dic_of_nodes_and_weights,list_of_weights=get_weights(input_nodes)
    All_weight = list_of_weights
    for sent_index, sentence in enumerate(S.values()): 
        (source_destinations,list_of_nodes) = sent_nodes[sent_index]
        nodes_weights = {}
        weights = []
        for node in list_of_nodes:
            nodes_weights[node] = dic_of_nodes_and_weights[node]
            weights.append(dic_of_nodes_and_weights[node])

        list_of_can_edges=get_candidate_edges(source_destinations,nodes_weights,weights,out_carteria,All_weight,dic_of_nodes_and_weights,All_SD)
        if len(list_of_can_edges)>=1:#we will make it a variable depends on the length of the sentence
            if(List_of_sent[sent_index] and (List_of_sent[sent_index].strip()) ):
                list_of_candidates_summeries.append(keyvalue[sent_index])
                list_pre_cansumm.append(List_of_sent[sent_index])
                

    global final_summary
    global visited_sentences
    max_words_limit= 70
    threshold = 15
    final_summary = []
    visited_sentences = []
    summary_length = 0
    candidate_summ = reorder(list_of_candidates_summeries)

    candidate_summ_len = 0
    for index,sent in enumerate(candidate_summ):
        candidate_summ_len = candidate_summ_len + get_words_count(sent)

    
    
    while(True):
        if(candidate_summ_len > (max_words_limit + threshold)):
            if(candidate_summ_len != last_summ_len):
                Cand_edge,list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges = iter_graph(Cand_edge,can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria) 

            elif(out_carteria != "max"):
                out_carteria="max"
                Cand_edge,list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges = iter_graph(Cand_edge,can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria)
            else:
                list_of_candidates_summeries = candidate_summ
                break
        elif(candidate_summ_len < (max_words_limit)):
            candidate_summ = Last_summ
            list_of_candidates_summeries = candidate_summ
            break
        else:
            list_of_candidates_summeries = candidate_summ
            break
    
    n_clusters,cluster_map = post_processing(list_of_candidates_summeries)
    final_summary = algorithm_4(summary_length,threshold,max_words_limit,list_of_candidates_summeries,n_clusters,cluster_map)
  
    
    final = reorder(final_summary)
    outsumm = concat(final)
    return outsumm
    

## to run the code 
## summary = summarization(input your text)